# Exercise 13

Some dimensionality reduction techniques can also be used for anomaly detection. For example, take the
Olivetti faces dataset, and reduce it with PCA, preserving 99% of the variance. Then compute the reconstruction
error for each image. Next, take some of the modified images you built in the previous exercise and
look at their reconstruction error: notice how much larger it is. If you plot a reconstructed image,
you will see why: it tries to reconstruct a normal face.

In [8]:
# imports
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np
from numpy.typing import NDArray

# functions
def determine_optimal_pca_components(
    X_train: NDArray[np.floating], variance_threshold: float = 0.95
) -> int:
    """
    Calculate the minimum number of principal components needed to retain a specified percentage of the variance.

    Args:
        X_train: 2D array-like, shape (n_samples, n_features), the training data.
        variance_threshold: Float between 0 and 1, the target fraction of variance to retain (default: 0.95).

    Returns:
        The minimum number of PCA components required to preserve at least the desired variance.
    """
    pca = PCA().fit(X_train)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    d = np.argmax(cumsum >= variance_threshold) + 1
    print(
        f"Optimal number of PCA components to keept at least variance {variance_threshold:.0%} is {d}"
    )
    return d

def get_modified_faces(
    X: NDArray[np.float64], y: NDArray[np.int32]
) -> tuple[NDArray[np.float64], NDArray[np.int32]]:
    """
    In accordance with the tasks from  exercises 12 and 13,  we modify some images (e.g. rotate, flip,
    darken)

    Args:
        X: Array of face images, each row is a flattened image of 64x64 pixels
        y: Array of labels corresponding to the face images

    Returns:
        X_bad_faces: Array of modified face images (rotated, flipped, darkened)
        y_bad: Array of labels for the modified images

    """
    # Re-using author's solution for this part (from: https://colab.research.google.com/github/ageron/handson-ml3/blob/main/09_unsupervised_learning.ipynb)
    n_rotated = 4
    rotated = np.transpose(X[:n_rotated].reshape(-1, 64, 64), axes=[0, 2, 1])
    rotated = rotated.reshape(-1, 64 * 64)
    y_rotated = y[:n_rotated]

    n_flipped = 3
    flipped = X[:n_flipped].reshape(-1, 64, 64)[:, ::-1]
    flipped = flipped.reshape(-1, 64 * 64)
    y_flipped = y[:n_flipped]

    n_darkened = 3
    darkened = X[:n_darkened].copy()
    darkened[:, 1:-1] *= 0.3
    y_darkened = y[:n_darkened]

    X_bad_faces = np.r_[rotated, flipped, darkened]
    y_bad = np.concatenate([y_rotated, y_flipped, y_darkened])
    return X_bad_faces, y_bad

## Step 1. Fetch and split the original dataset

In [2]:
olivetti = fetch_olivetti_faces()

In [3]:
X_train, _, y_train, _ = train_test_split(
    olivetti.data,
    olivetti.target,
    stratify=olivetti.target,
    test_size=0.2,
    random_state=42,
)

## Step 2. Reduce dataset's dimensionality with PCA

In [4]:
n_components = determine_optimal_pca_components(X_train, 0.99)
pca = PCA(n_components=n_components)
X_train_reduced = pca.fit_transform(X_train)

Optimal number of PCA components to keept at least variance 99% is 222


## Step 3. Compute the reconstruction error for each image

We can compute the reconstruction error for each image as the mean squared error (MSE) per image 
or the total squared error per image. The exercise doesn't specify, but since it's an image, we 
might compute the MSE.
  
We could equally use `sklearn.metrics.mean_squared_error`, but that would return us the average 
over all samples (images) and all features. Since we're asked for the error per image, we would 
have to use it in a loop, hence vectorized `np.mean(..., axis=1)` seems more efficient and straightforward.

Why `axis=1`? The array `X_train` has shape `(n_samples, n_features)`. When we compute `(X_train 
X_train_reconstructed) ** 2`, we get a matrix of squared errors of the same shape. Then, `np.mean(..., 
axis=1)` takes the mean across the columns (i.e., for each row, which represents one image). So 
we get one mean squared error per image.

In [6]:
X_train_reconstructed = pca.inverse_transform(X_train_reduced)
reconstruction_errors = np.mean(np.square(X_train - X_train_reconstructed), axis=1)

In [9]:
print(f"Train errors (AVG): {np.mean(reconstruction_errors)}")

Train errors (AVG): 0.00019272117060609162


## Step 4. Compute the reconstruction error for modified images

In [12]:
# Load a subset (10) of modified images
X_bad_faces, y_bad_faces = get_modified_faces(X_train, y_train)

In [13]:
X_bad_faces_reconstructed = pca.inverse_transform(pca.transform(X_bad_faces))
reconstruction_errors_modified = np.mean(np.square(X_bad_faces - X_bad_faces_reconstructed), axis=1)

In [14]:
print(f"Modified images' errors (AVG): {np.mean(reconstruction_errors_modified)}")

Modified images' errors (AVG): 0.004383236635476351
